# Secure sorting networks explained

In this notebook, we develop some MPC protocols for securely sorting lists of secret-shared numbers. Concretely, we will show how to define functions sorting lists of secure MPyC integers into ascending order. The values represented by the secure integers and their relative order should remain completely secret.

The explanation below assumes some basic familiarity with the MPyC framework for secure computation. Our main goal is to show how existing Python code for (oblivious) sorting can be used to implement a secure MPC sorting protocol using the `mpyc` package. The modifications to the existing code are very limited.

## Sorting networks

[Sorting networks](https://en.wikipedia.org/wiki/Sorting_network) are a classical type of comparison-based sorting algorithms. The basic operation (or, gate) in a sorting network is the *compare&swap* operation, which puts any two list elements $x[i]$ and $x[j]$, $i<j$, in ascending order. That is, only if $x[i]>x[j]$, elements $x[i]$ and $x[j]$ are swapped, and otherwise the compare&swap operation leaves the list unchanged. 

A sorting network specifies the exact sequence of compare&swap operations to be applied to a list of a given length $n$. The particular sequence depends only on $n$, the length of the input list. Even when the input list is already in ascending order, the sorting network will perform exactly as many---and actually the same---compare&swap operations as when the input list would be in descending order. 

For example, to sort a list of three numbers, one needs to perform three compare&swap operations with indices $(i,j)$ equal to $(0,1)$, then $(1,2)$, and finally once more $(0,1)$.

Below, we will use odd-even merge sort and bitonic sort, which are two well-known practical sorting networks. 

## MPyC setup

A simple MPyC setup using 32-bit (default) secure MPyC integers suffices for the purpose of this demonstration.

At this point we also import the Python `traceback` module for later use.

In [1]:
from mpyc.runtime import mpc    # load MPyC
secint = mpc.SecInt()           # 32-bit secure MPyC integers
mpc.run(mpc.start())            # required only when run with multiple parties
import traceback                # to show some suppressed error messages

2018-10-23 09:40:25,563 Start MPyC runtime v0.4.2


## Odd-even merge sort

Odd-even merge sort is an elegant, but somewhat intricate, sorting network. The details are nicely explained in the Wikipedia article [Batcher's Odd-Even Mergesort](https://en.wikipedia.org/wiki/Batcher_odd–even_mergesort). 

For our purposes, however, there is no need to understand exactly how this particular sorting network works. The only thing that we need to do is to grab the following  [example Python code](https://en.wikipedia.org/wiki/Batcher_odd–even_mergesort#Example_code) from this Wikipedia article.

In [2]:
def oddeven_merge(lo, hi, r):
    step = r * 2
    if step < hi - lo:
        yield from oddeven_merge(lo, hi, step)
        yield from oddeven_merge(lo + r, hi, step)
        yield from [(i, i + r) for i in range(lo + r, hi - r, step)]
    else:
        yield (lo, lo + r)

def oddeven_merge_sort_range(lo, hi):
    """ sort the part of x with indices between lo and hi.

    Note: endpoints (lo and hi) are included.
    """
    if (hi - lo) >= 1:
        # if there is more than one element, split the input
        # down the middle and first sort the first and second
        # half, followed by merging them.
        mid = lo + ((hi - lo) // 2)
        yield from oddeven_merge_sort_range(lo, mid)
        yield from oddeven_merge_sort_range(mid + 1, hi)
        yield from oddeven_merge(lo, hi, 1)

def oddeven_merge_sort(length):
    """ "length" is the length of the list to be sorted.
    Returns a list of pairs of indices starting with 0 """
    yield from oddeven_merge_sort_range(0, length - 1)

def compare_and_swap(x, a, b):
    if x[a] > x[b]:
        x[a], x[b] = x[b], x[a]

We run the code on a simple example. Note that this code assumes that the length of the input list is an integral power of two.

In [3]:
x = [2, 4, 3, 5, 6, 1, 7, 8]
for i in oddeven_merge_sort(len(x)): compare_and_swap(x, *i)
print(x)

[1, 2, 3, 4, 5, 6, 7, 8]


We try to run this code on a list of secure MPyC integers.

In [4]:
x = list(map(secint, [2, 4, 3, 5, 6, 1, 7, 8]))
try:
    for i in oddeven_merge_sort(len(x)): compare_and_swap(x, *i)
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-4-a875f9c118a5>", line 3, in <module>
    for i in oddeven_merge_sort(len(x)): compare_and_swap(x, *i)
  File "<ipython-input-2-a99b48746a99>", line 30, in compare_and_swap
    if x[a] > x[b]:
  File "c:\users\berry\appdata\local\programs\python\python36\lib\site-packages\mpyc-0.4.2-py3.6.egg\mpyc\sectypes.py", line 36, in __bool__
    raise TypeError('cannot use secure type in Boolean expressions')
TypeError: cannot use secure type in Boolean expressions


Unsurprisingly, this does not work. We get an error because we cannot use a `secint` directly in the condition of an `if` statement. And, even if we could, we should not do so, as the particular branch of the `if` statement followed reveals information about the input!

Therefore, the function `compare_and_swap` is modified (i) to hide whether elements of $x$ are swapped and (ii) to keep the values of the elements of $x$ hidden, even when these are swapped.

In [5]:
def compare_and_swap(x, a, b):
    c = x[a] > x[b]                  # secure comparison, secint c represents a secret-shared bit
    d = c * (x[b] - x[a])            # secure subtraction
    x[a], x[b] = x[a] + d, x[b] - d  # secure swap: x[a], x[b] swapped if only if c=1

Now the code can be used to sort a list of secure MPyC integers.

In [6]:
x = list(map(secint, [2, 4, 3, 5, 6, 1, 7, 8]))
for i in oddeven_merge_sort(len(x)): compare_and_swap(x, *i)
print(mpc.run(mpc.output(x)))

[1, 2, 3, 4, 5, 6, 7, 8]


## Bitonic sort

For our next example, we consult the Wikipedia article [Bitonic Sorter](https://en.wikipedia.org/wiki/Bitonic_sorter).

We apply the same approach, grabbing the [example Python code](https://en.wikipedia.org/wiki/Bitonic_sorter#Example_code) from the Wikipedia article, which is also designed to work for input lists whose length is an integral power of two.

In [7]:
def bitonic_sort(up, x):
    if len(x) <= 1:
        return x
    else: 
        first = bitonic_sort(True, x[:len(x) // 2])
        second = bitonic_sort(False, x[len(x) // 2:])
        return bitonic_merge(up, first + second)

def bitonic_merge(up, x): 
    # assume input x is bitonic, and sorted list is returned 
    if len(x) == 1:
        return x
    else:
        bitonic_compare(up, x)
        first = bitonic_merge(up, x[:len(x) // 2])
        second = bitonic_merge(up, x[len(x) // 2:])
        return first + second

def bitonic_compare(up, x):
    dist = len(x) // 2
    for i in range(dist):  
        if (x[i] > x[i + dist]) == up:
            x[i], x[i + dist] = x[i + dist], x[i] #swap

We run the code on the same example. 

In [8]:
print(bitonic_sort(True, [2, 4, 3, 5, 6, 1, 7, 8]))
print(bitonic_sort(False, [2, 4, 3, 5, 6, 1, 7, 8]))

[1, 2, 3, 4, 5, 6, 7, 8]
[8, 7, 6, 5, 4, 3, 2, 1]


Running the code on a list of secure MPyC integers gives the same error as above. 

In [9]:
x = list(map(secint, [2, 4, 3, 5, 6, 1, 7, 8]))
try:
    bitonic_sort(True, x)
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-9-4b604066647c>", line 3, in <module>
    bitonic_sort(True, x)
  File "<ipython-input-7-8a5c3e13ac49>", line 5, in bitonic_sort
    first = bitonic_sort(True, x[:len(x) // 2])
  File "<ipython-input-7-8a5c3e13ac49>", line 5, in bitonic_sort
    first = bitonic_sort(True, x[:len(x) // 2])
  File "<ipython-input-7-8a5c3e13ac49>", line 7, in bitonic_sort
    return bitonic_merge(up, first + second)
  File "<ipython-input-7-8a5c3e13ac49>", line 14, in bitonic_merge
    bitonic_compare(up, x)
  File "<ipython-input-7-8a5c3e13ac49>", line 22, in bitonic_compare
    if (x[i] > x[i + dist]) == up:
  File "c:\users\berry\appdata\local\programs\python\python36\lib\site-packages\mpyc-0.4.2-py3.6.egg\mpyc\sectypes.py", line 36, in __bool__
    raise TypeError('cannot use secure type in Boolean expressions')
TypeError: cannot use secure type in Boolean expressions


This time we modify the function `bitonic_compare` as follows again to hide what is happening to the elements of $x$ being compared.

In [10]:
def bitonic_compare(up, x):
    dist = len(x) // 2
    up = secint(up)                                    # convert public Boolean up into `secint` bit 
    for i in range(dist):
        b = (x[i] > x[i + dist]) ^ ~up                 # secure xor of comparison bit and negated up
        d = b * (x[i + dist] - x[i])                   # d = 0 or d = x[i + dist] - x[i]
        x[i], x[i + dist] = x[i] + d, x[i + dist] - d  # secure swap

Now the code can again be used to sort a list of secure MPyC integers.

In [11]:
print(mpc.run(mpc.output(bitonic_sort(True, x))))

[1, 2, 3, 4, 5, 6, 7, 8]


In [12]:
mpc.run(mpc.shutdown())   # required only when run with multiple parties

2018-10-23 09:40:26,468 Stop MPyC runtime -- elapsed time: 0:00:00.900694


The Python script [sort.py](sort.py) shows how to do secure bitonic sort for lists of arbitrary length, adapted from this general [bitonic sorter](http://www.iti.fh-flensburg.de/lang/algorithmen/sortieren/bitonic/oddn.htm).